In [65]:
import os
from astropy.table import Table as A_Table
from IPython.display import Image, display, clear_output
from astroquery.cadc import Cadc
from astropy.io.votable.tree import VOTableFile, Resource, Table, Field
from astropy.io.votable import from_table, writeto
import pandas as pd
import glue_jupyter as gj
import ipywidgets as widgets
from ipywidgets import interact
from cadcutils import net
from cadctap import CadcTapClient
from six import BytesIO



class Application:
    def __init__(self):
        self.data = pd.DataFrame()
        self.clicked_flag = 0
    
    #this function takes a query and parameters, if a output file is provided, then it will output the 
    # query result to the output_file, else return a astropy table object
    def get_cadc_data(self,query, param, output_file=None):
        cadc = Cadc()
        output = cadc.exec_sync(query.format(**param))
        if (output_file != None):
            self.__output_votable_file(output, output_file)
        else:
            return output
    #this function takes a astropy table and a .xml file as input,
    #returns a votable format and write
    def __output_votable_file(self, output, output_file):
        votable = from_table(output)
        writeto(votable, output_file)
        
    def get_cadcTap_data(self, query, fmt, out_file, tmp_table):
        anonSubject = net.Subject()
        #client = CadcTapClient(anonSubject, resource_id='ivo://cadc.nrc.ca/youcat')
        certSubject = net.Subject(certificate=os.path.join(os.environ['HOME'], ".ssl/cadcproxy.pem"))
        client = CadcTapClient(certSubject)
        client.query(query,
                     response_format=fmt,
                     output_file=out_file,
                     tmptable='tmptable:' + tmp_table,
                     timeout=30)
        self._process_out_data(out_file)
        
    def _process_out_data(self, file_name):
        data = pd.read_csv(file_name)
        cols = data.columns
        data.columns = [col.replace('"', '') for col in cols]
        data.dropna(subset=['Obs. ID'], inplace=True)
        data.to_csv(file_name, index=False)
        self.data = data
    
    def get_total_row_count(self):
        return len((self.data).index)
    
    def get_data(self):
        return self.data
    
    def start_glue(self, file_name):
        app = gj.jglue()
        glue_data = app.load_data(file_name)
        self.glue_data = glue_data
        self.app = app

        
    def get_scatter(self):
        scatter_viewer = self.app.scatter2d(x='Overall Quality',
                                       y='Really Bad Tracking',
                                       data=self.glue_data,
                                       show=True)
        return scatter_viewer
        
    def get_histogram(self):
        histogram_viewer = self.app.histogram1d(x='Really Bad Tracking',
                                           data=self.glue_data,
                                           show=True)
        return histogram_viewer
    
    def background_color(self,row):
        color = ''
        # hardcode 11 colours
        color_list = ['#F780BF', '#C7E6A1', '#A1E6E2', '#FAFBA1',
                  '#FF9C33', '#CBC6C0', '#D8A56E', '#F7A480',
                  '#D798EE', '#80A0C8', '#FACCED']
        for item in range(0, len(self.glue_data.subsets)):
            # Product ID is the 5th column
            if row.values[4] in self.glue_data.subsets[item]['Product ID']:
                color = color_list[item]
        return ['background-color: %s ' % color] * len(row.values)
    
    
    def get_subset(self, subset):
        lable = subset.split(' ')
        Id = int(lable[1]) - 1
        subset_data = {'Obs. ID': self.glue_data.subsets[Id]['Obs. ID'],
                       'publisherID': self.glue_data.subsets[Id]['Publisher ID']}
        subset_df = pd.DataFrame(subset_data)
        cadc = Cadc()
        subset_table = A_Table.from_pandas(subset_df)
        self.url_dictionary = {}
        for idx in range(len(subset_table)):
            # The get_data_urls only accepts a table as input
            url = cadc.get_data_urls(subset_table[idx: idx + 1],
                                 include_auxiliaries=True)
            fz_url = next((u for u in url if '.fz' in u), None)
            jpg_url = next((u for u in url if '1024.jpg' in u), None)
            # build the dictionary
            if jpg_url:
                self.url_dictionary[fz_url] = [jpg_url, subset_df['Obs. ID'][idx]]
        self.subset_df = subset_df
    

    def on_button_clicked(self,b):
        with self.out:
            clear_output()
            # when none of the buttons is clicked, display the entire table
            if self.clicked_flag == 0:
                display(self.data.style.apply(self.background_color,axis=1))
            else:
                #b.description to access the button 
                self.get_subset(b.description)
                display(self.subset_df)
        
    def get_table(self):
        self.button_clicked = 0;
        if (len(self.glue_data.subsets)):
            subset_option = []
            subset_num_records = []
            items_auto = []
            color_list = ['#F780BF', '#C7E6A1', '#A1E6E2', '#FAFBA1',
                          '#FF9C33', '#CBC6C0', '#D8A56E', '#F7A480',
                          '#D798EE', '#80A0C8', '#FACCED']
            for i in range(len(self.glue_data.subsets)):
                subset_option.append(self.glue_data.subsets[i].label)
                n_records = len(self.glue_data.subsets[i]['Product ID'])
                subset_num_records.append(n_records)
            for num in range(0, len(subset_option)):
                n_records = str(subset_num_records[num])on.on_click(on_button_clicked)
                bind
                t_tip = "number of records: {}".format(n_records)
                button = widgets.Button(description=subset_option[num],
                                        layout=widgets.Layout(flex='1 1 auto',
                                                              width='auto'),
                                        style=widgets.ButtonStyle(
                                            button_color=color_list[num]),
                                        tooltip=t_tip)
                ###
                self.out = widgets.Output()
                button.on_click(self.on_button_clicked)
                binded_button = widgets.VBox([button, self.out], layout=widgets.Layout(display='flex',
                                        flex_flow='row',
                                        align_items='stretch',
                                        width='100%'))
                items_auto.append(binded_button)
                ###
            #    items_auto.append(button)
            box_layout = widgets.Layout(display='flex',
                                        flex_flow='row',
                                        align_items='stretch',
                                        width='100%')
            box_auto = widgets.Box(children=items_auto,
                                   layout=box_layout)
            display(box_auto)
            if( self.clicked_flag != 1 ):
                button.click()
                self.clicked_flag = 1
        else:
            display(self.data)
    
    
        


####test class###
application = Application()
observation_id = widgets.IntText(value=1013372,
                                 description='Observation ID',
                                 continuous_update=False,
                                 style={'description_width': 'initial'})
ui_database = widgets.HBox([observation_id])

G_Type = widgets.Dropdown(options=['Scatter', 'Histogram', 'Table'],
                          description='View Data',
                          style={'description_width': 'initial'})
ui_graph = widgets.HBox([G_Type])


def graphs(G_Type):
    clicked_flag = 0
    if (G_Type == 'Scatter'):
        application.get_scatter()

    elif (G_Type == 'Histogram'):
        application.get_histogram()
    else:
        application.get_table()
    


def retrieve_data(observation_id):
    obs_query = """SELECT Observation.observationURI,
    Plane.publisherID,
    Observation.collection,
    Observation.observationID,
    Plane.productID,
    Observation.instrument_name,
    Plane.time_exposure
    FROM caom2.Plane AS Plane
    JOIN caom2.Observation AS Observation
    ON Plane.obsID = Observation.obsID
    WHERE (Observation.observationID = '{obs_id}'
    AND collection = '{collection}'
    AND instrument_name = '{instrument}')"""
    obs_query_param = {'obs_id': observation_id,
                       'collection': 'CFHT',
                       'instrument': 'MegaPrime'}
    outfile = 'tmp/output.xml'
    application.get_cadc_data(obs_query,obs_query_param,output_file=outfile)
    
    quality_query = """SELECT Tmp.observationURI AS Preview,
        Tmp.publisherID AS "Publisher ID",
        Tmp.collection AS Collection,
        Tmp.observationID AS "Obs. ID",
        Tmp.productID AS "Product ID",
        Tmp.instrument_name AS Instrument,
        Tmp.time_exposure AS "Int. Time",
        Quality.overallQuality AS "Overall Quality",
        Quality.reallyBadTracking AS "Really Bad Tracking",
        Quality.badTracking AS "Bad Tracking",
        Quality.badWeather AS "Bad Weather",
        Quality.backgroundProblem AS "Background Problem",
        Quality.deadCCDs AS "Dead CCDs"
        FROM ml.MegaprimeQuality AS Quality
        JOIN tap_upload.tmptable AS Tmp
        ON Quality.observationID=Tmp.observationID
        WHERE publisherID LIKE '%p'""" 
    tmp_table = os.path.join('tmp', 'output.xml')
    application.get_cadcTap_data(quality_query, 'csv', 'tmp/output_file.csv', tmp_table)
    application.get_total_row_count()
    print('Total row count: {}'.format(str(application.get_total_row_count())))
    application.start_glue('tmp/output_file.csv')
    graph_output = widgets.interactive_output(graphs, {'G_Type': G_Type})
    display(ui_graph, graph_output)
    
    

        
database_output = widgets.interactive_output(retrieve_data, {"observation_id": observation_id})
display(ui_database , database_output)        

Output()